In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
import logging

In [2]:
log_fmt = "[%(asctime)s] %(levelname)s in %(module)s: %(message)s"
logging.basicConfig(format=log_fmt, level=logging.INFO)

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
train_embed = pd.read_csv('../data2/train_embed_1205_1.txt', sep='\t')
train_embed.head()

,问题id,用户id,邀请创建时间,是否回答,访问频率,用户二分类特征a,用户二分类特征b,用户二分类特征c,用户二分类特征d,用户二分类特征e,...,用户多分类特征b,用户多分类特征c,用户多分类特征d,用户多分类特征e,盐值,focus_topics,question_topics,title_questions,describe_questions,interest_topic
0,Q2166419046,M401693808,D3865-H22,0,3.0,0,1,0,0,0,...,160,244,787,1,297,[ 0.56766598 -0.29046254 0.4245674 1.052861...,[-1.234756 1.88610096 -1.81519779 1.470050...,[-1.06652634 2.15964371 -2.12913373 1.306371...,[-0.01129505 3.07473125 -0.96356895 -1.014970...,[-0.26751407 -1.18381493 0.63383315 0.594649...
1,Q1550017551,M3392373099,D3844-H11,0,2.0,1,0,0,0,0,...,109,249,647,0,308,[ 1.26680035 1.00163364 -1.30190022 -0.003696...,[ 1.27062557 -2.93239985 -0.18030047 2.482307...,[-1.73886358e+00 -4.31728346e+00 -2.79413069e+...,[-0.19891346 -1.41523748 -0.05533847 0.637761...,[ 0.4099659 -1.29397761 -0.60552278 1.103454...
2,Q604029601,M2317670257,D3862-H15,0,3.0,1,0,0,0,0,...,193,0,433,1,415,[ 0.32695651 -0.46650439 0.3626379 0.701971...,[ 1.11285402 0.38576528 -1.29714541 -0.199438...,[ 2.36547675 2.25407476 -5.04879473 2.117108...,[ 0.26690856 -0.83421576 -2.30800738 2.237093...,[ 1.59588447 -3.68648277 -1.00653933 0.097648...
3,Q2350061229,M1618461867,D3849-H11,0,4.0,1,0,0,1,0,...,109,244,787,1,305,[ 0.24529339 0.00517115 0.01876875 0.898242...,[-1.3105482 -0.1839075 1.5894188 1.145820...,[ 1.7495643 1.59935852 -0.58167021 -2.834890...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-0.72625533 0.60840047 -0.00724587 0.090645...
4,Q2443223942,M3544409350,D3867-H4,0,2.0,1,0,0,0,0,...,191,175,695,1,296,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[ 2.27136827e+00 -1.88522457e+00 -1.38685803e+...,[ 0.78020852 -1.81377295 -1.51680944 0.321364...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[ 0.79694882 0.73822912 0.17052762 -1.028728...


In [5]:
train_embed.shape

(37145, 21)

In [6]:
test_embed = pd.read_csv('../data2/test_embed_1205_1.txt',sep='\t')
test_embed.head()

,问题id,用户id,验证创建时间,访问频率,用户二分类特征a,用户二分类特征b,用户二分类特征c,用户二分类特征d,用户二分类特征e,用户多分类特征a,用户多分类特征b,用户多分类特征c,用户多分类特征d,用户多分类特征e,盐值,focus_topics,question_topics,title_questions,describe_questions,interest_topic
0,Q1493039281,M64135255,D3870-H9,3.0,0,1,0,0,0,871,89,165,915,1,370,[ 0.61519237 -0.71950003 -0.89804152 0.943996...,[ 0.44677012 -1.26927877 -2.08104793 -0.254532...,[ 0.94767582 -0.01583392 -3.45240414 -2.027137...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-0.69987991 -1.1132572 -2.22383697 1.277995...
1,Q2023398782,M2536956560,D3872-H22,4.0,0,1,0,1,0,871,89,321,345,1,291,[ 1.31995794 -0.65841173 -0.40665465 0.167623...,[-1.4924337 -2.2450113 -3.7260833 -1.6556 ...,[-0.9866205 -1.48297908 -5.22400575 1.352380...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-0.56391784 0.58273743 -1.05411074 -0.064027...
2,Q4151338694,M3294926344,D3874-H15,3.0,1,0,0,0,0,871,89,167,295,1,451,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[ 2.4026921 -3.8089825 -2.75084226 0.827597...,[ 4.45145923 -3.74320863 -2.09977811 1.178978...,[ 3.40120825 -2.79011562 -2.87315787 0.631374...,[ 0.44810855 -1.39813822 1.00532829 -0.255647...
3,Q3271436624,M3744310794,D3873-H4,NaN,0,0,0,0,0,871,89,151,699,0,416,[ 1.21585152 -0.77842913 -0.58827993 -2.646894...,[ 2.00385745 1.9272432 -1.61824365 2.452354...,[ 0.72658553 -0.6225797 -2.16017961 -0.273752...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[ 0.33681995 1.52773287 -0.81048186 -0.719526...
4,Q3314287018,M1349051752,D3872-H19,3.0,0,1,0,0,0,282,132,271,535,1,316,[ 0.51993473 -0.25766414 -0.13897653 0.471269...,[ 0.7670328 -1.7355047 0.8811254 -0.534188...,[-1.33332375 -1.44937158 0.59044838 -1.751932...,[ 0.09084882 -0.83723214 -1.73629461 -0.277638...,[-1.36549600e-01 -1.84660976e+00 -6.65001650e-...


In [7]:
test_embed.shape

(1141683, 20)

In [8]:
def extract_day(s):
    return s.apply(lambda x: int(x.split('-')[0][1:]))

def extract_hour(s):
    return s.apply(lambda x: int(x.split('-')[1][1:]))

In [9]:
train_embed['day'] =extract_day( train_embed['邀请创建时间'])
train_embed['hour'] = extract_hour(train_embed['邀请创建时间'])

In [10]:
base_path = '../data'

In [11]:
# test = pd.read_csv(f'{base_path}/invite_info_evaluate_1_0926.txt', sep='\t', header=None)
# test.columns = ['qid', 'uid', 'dt']
# logging.info("test %s", test.shape)

# sub = test.copy()

# sub_size = len(sub)
# test['day'] = extract_day(test['dt'])
# test['hour'] = extract_hour(test['dt'])
# del test['dt']

In [12]:
train_embed.columns = ['qid','uid','cre_time','is_ans','freq','uf_b1','uf_b2','uf_b3','uf_b4','uf_b5','uf_c1','uf_c2','uf_c3','uf_c4','uf_c5','scores','focus_topics','question_topics','title_questions','describe_questions','interest_topic','day','hour']
del train_embed['cre_time']

In [13]:
train_embed.head()

,qid,uid,is_ans,freq,uf_b1,uf_b2,uf_b3,uf_b4,uf_b5,uf_c1,...,uf_c4,uf_c5,scores,focus_topics,question_topics,title_questions,describe_questions,interest_topic,day,hour
0,Q2166419046,M401693808,0,3.0,0,1,0,0,0,1864,...,787,1,297,[ 0.56766598 -0.29046254 0.4245674 1.052861...,[-1.234756 1.88610096 -1.81519779 1.470050...,[-1.06652634 2.15964371 -2.12913373 1.306371...,[-0.01129505 3.07473125 -0.96356895 -1.014970...,[-0.26751407 -1.18381493 0.63383315 0.594649...,3865,22
1,Q1550017551,M3392373099,0,2.0,1,0,0,0,0,1065,...,647,0,308,[ 1.26680035 1.00163364 -1.30190022 -0.003696...,[ 1.27062557 -2.93239985 -0.18030047 2.482307...,[-1.73886358e+00 -4.31728346e+00 -2.79413069e+...,[-0.19891346 -1.41523748 -0.05533847 0.637761...,[ 0.4099659 -1.29397761 -0.60552278 1.103454...,3844,11
2,Q604029601,M2317670257,0,3.0,1,0,0,0,0,1347,...,433,1,415,[ 0.32695651 -0.46650439 0.3626379 0.701971...,[ 1.11285402 0.38576528 -1.29714541 -0.199438...,[ 2.36547675 2.25407476 -5.04879473 2.117108...,[ 0.26690856 -0.83421576 -2.30800738 2.237093...,[ 1.59588447 -3.68648277 -1.00653933 0.097648...,3862,15
3,Q2350061229,M1618461867,0,4.0,1,0,0,1,0,1065,...,787,1,305,[ 0.24529339 0.00517115 0.01876875 0.898242...,[-1.3105482 -0.1839075 1.5894188 1.145820...,[ 1.7495643 1.59935852 -0.58167021 -2.834890...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-0.72625533 0.60840047 -0.00724587 0.090645...,3849,11
4,Q2443223942,M3544409350,0,2.0,1,0,0,0,0,499,...,695,1,296,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[ 2.27136827e+00 -1.88522457e+00 -1.38685803e+...,[ 0.78020852 -1.81377295 -1.51680944 0.321364...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[ 0.79694882 0.73822912 0.17052762 -1.028728...,3867,4


In [14]:
test_embed['day'] =extract_day( test_embed['验证创建时间'])
test_embed['hour'] = extract_hour(test_embed['验证创建时间'])

In [15]:
test_embed.columns = ['qid','uid','cre_time','freq','uf_b1','uf_b2','uf_b3','uf_b4','uf_b5','uf_c1','uf_c2','uf_c3','uf_c4','uf_c5','scores','focus_topics','question_topics','title_questions','describe_questions','interest_topic','day','hour']
del test_embed['cre_time']

In [16]:
test_embed.head()

,qid,uid,freq,uf_b1,uf_b2,uf_b3,uf_b4,uf_b5,uf_c1,uf_c2,...,uf_c4,uf_c5,scores,focus_topics,question_topics,title_questions,describe_questions,interest_topic,day,hour
0,Q1493039281,M64135255,3.0,0,1,0,0,0,871,89,...,915,1,370,[ 0.61519237 -0.71950003 -0.89804152 0.943996...,[ 0.44677012 -1.26927877 -2.08104793 -0.254532...,[ 0.94767582 -0.01583392 -3.45240414 -2.027137...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-0.69987991 -1.1132572 -2.22383697 1.277995...,3870,9
1,Q2023398782,M2536956560,4.0,0,1,0,1,0,871,89,...,345,1,291,[ 1.31995794 -0.65841173 -0.40665465 0.167623...,[-1.4924337 -2.2450113 -3.7260833 -1.6556 ...,[-0.9866205 -1.48297908 -5.22400575 1.352380...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-0.56391784 0.58273743 -1.05411074 -0.064027...,3872,22
2,Q4151338694,M3294926344,3.0,1,0,0,0,0,871,89,...,295,1,451,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[ 2.4026921 -3.8089825 -2.75084226 0.827597...,[ 4.45145923 -3.74320863 -2.09977811 1.178978...,[ 3.40120825 -2.79011562 -2.87315787 0.631374...,[ 0.44810855 -1.39813822 1.00532829 -0.255647...,3874,15
3,Q3271436624,M3744310794,NaN,0,0,0,0,0,871,89,...,699,0,416,[ 1.21585152 -0.77842913 -0.58827993 -2.646894...,[ 2.00385745 1.9272432 -1.61824365 2.452354...,[ 0.72658553 -0.6225797 -2.16017961 -0.273752...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[ 0.33681995 1.52773287 -0.81048186 -0.719526...,3873,4
4,Q3314287018,M1349051752,3.0,0,1,0,0,0,282,132,...,535,1,316,[ 0.51993473 -0.25766414 -0.13897653 0.471269...,[ 0.7670328 -1.7355047 0.8811254 -0.534188...,[-1.33332375 -1.44937158 0.59044838 -1.751932...,[ 0.09084882 -0.83723214 -1.73629461 -0.277638...,[-1.36549600e-01 -1.84660976e+00 -6.65001650e-...,3872,19


In [17]:
# 加载问题
ques = pd.read_csv(f'{base_path}/question_info_0926.txt', header=None, sep='\t')
ques.columns = ['qid', 'q_dt', 'title_t1', 'title_t2', 'desc_t1', 'desc_t2', 'topic']
del ques['title_t1'], ques['title_t2'], ques['desc_t1'], ques['desc_t2']
logging.info("ques %s", ques.shape)

ques['q_day'] = extract_day(ques['q_dt'])
ques['q_hour'] = extract_hour(ques['q_dt'])
del ques['q_dt']

[2019-12-08 18:13:59,213] INFO in <ipython-input-17-d1d53554f000>: ques (1829900, 3)


In [18]:
ques.head()

,qid,topic,q_day,q_hour
0,Q2234111670,"T321,T730,T5784,T4389",1018,5
1,Q760329790,"T278,T12673,T4677",1745,20
2,Q741313548,T226,2032,21
3,Q3481466230,"T51,T4468",2185,15
4,Q3966197028,"T54700,T81,T57,T17670,T43574",2269,17


In [19]:
# 加载回答
ans = pd.read_csv(f'{base_path}/answer_info_0926.txt', header=None, sep='\t')
ans.columns = ['aid', 'qid', 'uid', 'ans_dt', 'ans_t1', 'ans_t2', 'is_good', 'is_rec', 'is_dest', 'has_img',
               'has_video', 'word_count', 'reci_cheer', 'reci_uncheer', 'reci_comment', 'reci_mark', 'reci_tks',
               'reci_xxx', 'reci_no_help', 'reci_dis']
del ans['ans_t1'], ans['ans_t2']
logging.info("ans %s", ans.shape)

ans['a_day'] = extract_day(ans['ans_dt'])
ans['a_hour'] = extract_hour(ans['ans_dt'])
del ans['ans_dt']

ans = pd.merge(ans, ques, on='qid')

[2019-12-08 18:17:16,442] INFO in <ipython-input-19-215a7a999321>: ans (4513735, 18)


In [44]:
print(ans['a_day'].max())
print(ans['a_day'].min())

3867
3807


In [20]:
# 回答距提问的天数
ans['diff_qa_days'] = ans['a_day'] - ans['q_day']

[2019-12-08 18:18:00,805] INFO in utils: Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[2019-12-08 18:18:00,807] INFO in utils: NumExpr defaulting to 8 threads.


In [21]:
ans.head()

,aid,qid,uid,is_good,is_rec,is_dest,has_img,has_video,word_count,reci_cheer,...,reci_tks,reci_xxx,reci_no_help,reci_dis,a_day,a_hour,topic,q_day,q_hour,diff_qa_days
0,A2502060945,Q1867533817,M625498202,0,0,0,0,0,41,1,...,1,0,0,0,3808,7,"T381,T8211,T3144,T4936,T823",3806,21,2
1,A247365975,Q1867533817,M279074599,0,0,0,0,0,123,0,...,0,0,0,1,3821,9,"T381,T8211,T3144,T4936,T823",3806,21,15
2,A2661909364,Q1867533817,M3094956744,0,0,0,0,0,27,1,...,3,0,0,0,3808,5,"T381,T8211,T3144,T4936,T823",3806,21,2
3,A2881302290,Q1867533817,M2926469058,0,0,0,0,0,144,0,...,0,0,0,1,3821,13,"T381,T8211,T3144,T4936,T823",3806,21,15
4,A2847829478,Q3366788616,M142330444,0,0,0,0,0,204,1,...,1,0,0,0,3810,17,"T5490,T2180,T17098",3808,9,2


In [22]:
train = train_embed
train['label']=train['is_ans']
del train['is_ans']

In [23]:
test = test_embed

In [24]:
# 时间窗口划分
# train
# val
train_start = 3838
train_end = 3867

val_start = 3868
val_end = 3874

label_end = 3867
label_start = label_end - 6

train_label_feature_end = label_end - 7
train_label_feature_start = train_label_feature_end - 22

train_ans_feature_end = label_end - 7
train_ans_feature_start = train_ans_feature_end - 50

val_label_feature_end = val_start - 1
val_label_feature_start = val_label_feature_end - 22

val_ans_feature_end = val_start - 1
val_ans_feature_start = val_ans_feature_end - 50

train_label_feature = train[(train['day'] >= train_label_feature_start) & (train['day'] <= train_label_feature_end)]
logging.info("train_label_feature %s", train_label_feature.shape)

val_label_feature = train[(train['day'] >= val_label_feature_start) & (train['day'] <= val_label_feature_end)]
logging.info("val_label_feature %s", val_label_feature.shape)

train_label = train[(train['day'] > train_label_feature_end)]

logging.info("train feature start %s end %s, label start %s end %s", train_label_feature['day'].min(),
             train_label_feature['day'].max(), train_label['day'].min(), train_label['day'].max())

logging.info("test feature start %s end %s, label start %s end %s", val_label_feature['day'].min(),
             val_label_feature['day'].max(), test['day'].min(), test['day'].max())

[2019-12-08 18:18:01,337] INFO in <ipython-input-24-413cbf6f4a7d>: train_label_feature (26950, 22)
[2019-12-08 18:18:01,366] INFO in <ipython-input-24-413cbf6f4a7d>: val_label_feature (29711, 22)
[2019-12-08 18:18:01,377] INFO in <ipython-input-24-413cbf6f4a7d>: train feature start 3838 end 3860, label start 3861 end 3867
[2019-12-08 18:18:01,398] INFO in <ipython-input-24-413cbf6f4a7d>: test feature start 3845 end 3867, label start 3868 end 3874


In [25]:
# 确定ans的时间范围
# 3807~3874
train_ans_feature = ans[(ans['a_day'] >= train_ans_feature_start) & (ans['a_day'] <= train_ans_feature_end)]

val_ans_feature = ans[(ans['a_day'] >= val_ans_feature_start) & (ans['a_day'] <= val_ans_feature_end)]

logging.info("train ans feature %s, start %s end %s", train_ans_feature.shape, train_ans_feature['a_day'].min(),
             train_ans_feature['a_day'].max())

logging.info("val ans feature %s, start %s end %s", val_ans_feature.shape, val_ans_feature['a_day'].min(),
             val_ans_feature['a_day'].max())

fea_cols = ['is_good', 'is_rec', 'is_dest', 'has_img', 'has_video', 'word_count',
            'reci_cheer', 'reci_uncheer', 'reci_comment', 'reci_mark', 'reci_tks',
            'reci_xxx', 'reci_no_help', 'reci_dis', 'diff_qa_days']

[2019-12-08 18:18:16,024] INFO in <ipython-input-25-aa15f72a2344>: train ans feature (3700178, 23), start 3810 end 3860
[2019-12-08 18:18:16,068] INFO in <ipython-input-25-aa15f72a2344>: val ans feature (3992334, 23), start 3817 end 3867


In [26]:
def extract_feature1(target, label_feature, ans_feature):
#     # 问题特征
#     t1 = label_feature.groupby('qid')['label'].agg(['mean', 'sum', 'std', 'count']).reset_index()
#     t1.columns = ['qid', 'q_inv_mean', 'q_inv_sum', 'q_inv_std', 'q_inv_count']
#     target = pd.merge(target, t1, on='qid', how='left')

#     # 用户特征
#     t1 = label_feature.groupby('uid')['label'].agg(['mean', 'sum', 'std', 'count']).reset_index()
#     t1.columns = ['uid', 'u_inv_mean', 'u_inv_sum', 'u_inv_std', 'u_inv_count']
#     target = pd.merge(target, t1, on='uid', how='left')
    #
    # train_size = len(train)
    # data = pd.concat((train, test), sort=True)

    # 回答部分特征

    t1 = ans_feature.groupby('qid')['aid'].count().reset_index()
    t1.columns = ['qid', 'q_ans_count']
    target = pd.merge(target, t1, on='qid', how='left')

    t1 = ans_feature.groupby('uid')['aid'].count().reset_index()
    t1.columns = ['uid', 'u_ans_count']
    target = pd.merge(target, t1, on='uid', how='left')

    for col in fea_cols:
        t1 = ans_feature.groupby('uid')[col].agg(['sum', 'max', 'mean']).reset_index()
        t1.columns = ['uid', f'u_{col}_sum', f'u_{col}_max', f'u_{col}_mean']
        target = pd.merge(target, t1, on='uid', how='left')

        t1 = ans_feature.groupby('qid')[col].agg(['sum', 'max', 'mean']).reset_index()
        t1.columns = ['qid', f'q_{col}_sum', f'q_{col}_max', f'q_{col}_mean']
        target = pd.merge(target, t1, on='qid', how='left')
        logging.info("extract %s", col)
    return target

In [27]:
train_label = extract_feature1(train_label, train_label_feature, train_ans_feature)
test = extract_feature1(test, val_label_feature, val_ans_feature)

[2019-12-08 18:19:06,324] INFO in <ipython-input-26-a2b3ea27ea45>: extract is_good
[2019-12-08 18:19:17,812] INFO in <ipython-input-26-a2b3ea27ea45>: extract is_rec
[2019-12-08 18:19:28,428] INFO in <ipython-input-26-a2b3ea27ea45>: extract is_dest
[2019-12-08 18:19:39,623] INFO in <ipython-input-26-a2b3ea27ea45>: extract has_img
[2019-12-08 18:19:54,948] INFO in <ipython-input-26-a2b3ea27ea45>: extract has_video
[2019-12-08 18:20:05,695] INFO in <ipython-input-26-a2b3ea27ea45>: extract word_count
[2019-12-08 18:20:17,990] INFO in <ipython-input-26-a2b3ea27ea45>: extract reci_cheer
[2019-12-08 18:20:29,751] INFO in <ipython-input-26-a2b3ea27ea45>: extract reci_uncheer
[2019-12-08 18:20:41,611] INFO in <ipython-input-26-a2b3ea27ea45>: extract reci_comment
[2019-12-08 18:20:53,234] INFO in <ipython-input-26-a2b3ea27ea45>: extract reci_mark
[2019-12-08 18:21:04,940] INFO in <ipython-input-26-a2b3ea27ea45>: extract reci_tks
[2019-12-08 18:21:15,308] INFO in <ipython-input-26-a2b3ea27ea45>: 

In [28]:
train_label.head()

,qid,uid,freq,uf_b1,uf_b2,uf_b3,uf_b4,uf_b5,uf_c1,uf_c2,...,u_reci_dis_mean,q_reci_dis_sum,q_reci_dis_max,q_reci_dis_mean,u_diff_qa_days_sum,u_diff_qa_days_max,u_diff_qa_days_mean,q_diff_qa_days_sum,q_diff_qa_days_max,q_diff_qa_days_mean
0,Q2166419046,M401693808,3.0,0,1,0,0,0,1864,160,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Q604029601,M2317670257,3.0,1,0,0,0,0,1347,193,...,0.000000,NaN,NaN,NaN,2.0,2.0,1.000000,NaN,NaN,NaN
2,Q2443223942,M3544409350,2.0,1,0,0,0,0,499,191,...,NaN,0.0,0.0,0.0,NaN,NaN,NaN,305.0,13.0,9.531250
3,Q795459266,M2818659842,4.0,1,0,0,0,0,1347,193,...,0.000000,0.0,0.0,0.0,2260.0,1391.0,376.666667,667.0,234.0,222.333333
4,Q110462128,M848334644,3.0,1,0,0,0,0,1905,26,...,0.017857,NaN,NaN,NaN,2156.0,1426.0,38.500000,NaN,NaN,NaN


In [29]:
test.head()

,qid,uid,freq,uf_b1,uf_b2,uf_b3,uf_b4,uf_b5,uf_c1,uf_c2,...,u_reci_dis_mean,q_reci_dis_sum,q_reci_dis_max,q_reci_dis_mean,u_diff_qa_days_sum,u_diff_qa_days_max,u_diff_qa_days_mean,q_diff_qa_days_sum,q_diff_qa_days_max,q_diff_qa_days_mean
0,Q1493039281,M64135255,3.0,0,1,0,0,0,871,89,...,0.0,NaN,NaN,NaN,1087.0,1085.0,543.50,NaN,NaN,NaN
1,Q2023398782,M2536956560,4.0,0,1,0,1,0,871,89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Q4151338694,M3294926344,3.0,1,0,0,0,0,871,89,...,8.0,NaN,NaN,NaN,1125.0,694.0,281.25,NaN,NaN,NaN
3,Q3271436624,M3744310794,NaN,0,0,0,0,0,871,89,...,0.0,0.0,0.0,0.0,807.0,807.0,807.00,12.0,11.0,3.0
4,Q3314287018,M1349051752,3.0,0,1,0,0,0,282,132,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
train_label.columns.values.tolist()

['qid',
 'uid',
 'freq',
 'uf_b1',
 'uf_b2',
 'uf_b3',
 'uf_b4',
 'uf_b5',
 'uf_c1',
 'uf_c2',
 'uf_c3',
 'uf_c4',
 'uf_c5',
 'scores',
 'focus_topics',
 'question_topics',
 'title_questions',
 'describe_questions',
 'interest_topic',
 'day',
 'hour',
 'label',
 'q_ans_count',
 'u_ans_count',
 'u_is_good_sum',
 'u_is_good_max',
 'u_is_good_mean',
 'q_is_good_sum',
 'q_is_good_max',
 'q_is_good_mean',
 'u_is_rec_sum',
 'u_is_rec_max',
 'u_is_rec_mean',
 'q_is_rec_sum',
 'q_is_rec_max',
 'q_is_rec_mean',
 'u_is_dest_sum',
 'u_is_dest_max',
 'u_is_dest_mean',
 'q_is_dest_sum',
 'q_is_dest_max',
 'q_is_dest_mean',
 'u_has_img_sum',
 'u_has_img_max',
 'u_has_img_mean',
 'q_has_img_sum',
 'q_has_img_max',
 'q_has_img_mean',
 'u_has_video_sum',
 'u_has_video_max',
 'u_has_video_mean',
 'q_has_video_sum',
 'q_has_video_max',
 'q_has_video_mean',
 'u_word_count_sum',
 'u_word_count_max',
 'u_word_count_mean',
 'q_word_count_sum',
 'q_word_count_max',
 'q_word_count_mean',
 'u_reci_cheer_sum',
 

In [31]:
test.columns.values.tolist()

['qid',
 'uid',
 'freq',
 'uf_b1',
 'uf_b2',
 'uf_b3',
 'uf_b4',
 'uf_b5',
 'uf_c1',
 'uf_c2',
 'uf_c3',
 'uf_c4',
 'uf_c5',
 'scores',
 'focus_topics',
 'question_topics',
 'title_questions',
 'describe_questions',
 'interest_topic',
 'day',
 'hour',
 'q_ans_count',
 'u_ans_count',
 'u_is_good_sum',
 'u_is_good_max',
 'u_is_good_mean',
 'q_is_good_sum',
 'q_is_good_max',
 'q_is_good_mean',
 'u_is_rec_sum',
 'u_is_rec_max',
 'u_is_rec_mean',
 'q_is_rec_sum',
 'q_is_rec_max',
 'q_is_rec_mean',
 'u_is_dest_sum',
 'u_is_dest_max',
 'u_is_dest_mean',
 'q_is_dest_sum',
 'q_is_dest_max',
 'q_is_dest_mean',
 'u_has_img_sum',
 'u_has_img_max',
 'u_has_img_mean',
 'q_has_img_sum',
 'q_has_img_max',
 'q_has_img_mean',
 'u_has_video_sum',
 'u_has_video_max',
 'u_has_video_mean',
 'q_has_video_sum',
 'q_has_video_max',
 'q_has_video_mean',
 'u_word_count_sum',
 'u_word_count_max',
 'u_word_count_mean',
 'q_word_count_sum',
 'q_word_count_max',
 'q_word_count_mean',
 'u_reci_cheer_sum',
 'u_reci_ch

In [32]:
test.shape

(1141683, 113)

In [33]:
testdd = pd.read_csv(f'{base_path}/invite_info_evaluate_1_0926.txt', sep='\t', header=None)
testdd.columns = ['qid', 'uid', 'dt']
logging.info("test %s", testdd.shape)

sub = testdd.copy()

sub_size = len(sub)

[2019-12-08 18:25:47,887] INFO in <ipython-input-33-7f6828fe47f0>: test (1141683, 3)


In [34]:
# 特征提取结束
logging.info("train shape %s, test shape %s", train_label.shape, test.shape)
assert len(test) == sub_size

# 加载用户
user = pd.read_csv(f'{base_path}/member_info_0926.txt', header=None, sep='\t')
user.columns = ['uid', 'gender', 'creat_keyword', 'level', 'hot', 'reg_type', 'reg_plat', 'freq', 'uf_b1', 'uf_b2',
                'uf_b3', 'uf_b4', 'uf_b5', 'uf_c1', 'uf_c2', 'uf_c3', 'uf_c4', 'uf_c5', 'score', 'follow_topic',
                'inter_topic']
del user['follow_topic'], user['inter_topic']
logging.info("user %s", user.shape)

unq = user.nunique()
logging.info("user unq %s", unq)

for x in unq[unq == 1].index:
    del user[x]
    logging.info('del unq==1 %s', x)

t = user.dtypes
cats = [x for x in t[t == 'object'].index if x not in ['follow_topic', 'inter_topic', 'uid']]
logging.info("user cat %s", cats)

for d in cats:
    lb = LabelEncoder()
    user[d] = lb.fit_transform(user[d])
    logging.info('encode %s', d)

q_lb = LabelEncoder()
q_lb.fit(list(train_label['qid'].astype(str).values) + list(test['qid'].astype(str).values))
train_label['qid_enc'] = q_lb.transform(train_label['qid'])
test['qid_enc'] = q_lb.transform(test['qid'])

u_lb = LabelEncoder()
u_lb.fit(user['uid'])
train_label['uid_enc'] = u_lb.transform(train_label['uid'])
test['uid_enc'] = u_lb.transform(test['uid'])

# merge user
# train_label = pd.merge(train_label, user, on='uid', how='left')
# test = pd.merge(test, user, on='uid', how='left')
logging.info("train shape %s, test shape %s", train_label.shape, test.shape)

data = pd.concat((train_label, test), axis=0, sort=True)

[2019-12-08 18:25:48,016] INFO in <ipython-input-34-48ea3629568f>: train shape (10195, 114), test shape (1141683, 113)
[2019-12-08 18:26:01,202] INFO in <ipython-input-34-48ea3629568f>: user (1931654, 19)
[2019-12-08 18:26:05,589] INFO in <ipython-input-34-48ea3629568f>: user unq uid              1931654
gender                 3
creat_keyword          1
level                  1
hot                    1
reg_type               1
reg_plat               1
freq                   5
uf_b1                  2
uf_b2                  2
uf_b3                  2
uf_b4                  2
uf_b5                  2
uf_c1               2561
uf_c2                291
uf_c3                428
uf_c4               1556
uf_c5                  2
score                732
dtype: int64
[2019-12-08 18:26:05,633] INFO in <ipython-input-34-48ea3629568f>: del unq==1 creat_keyword
[2019-12-08 18:26:05,641] INFO in <ipython-input-34-48ea3629568f>: del unq==1 level
[2019-12-08 18:26:05,643] INFO in <ipython-input-34-48e

In [35]:
train_label.head()

,qid,uid,freq,uf_b1,uf_b2,uf_b3,uf_b4,uf_b5,uf_c1,uf_c2,...,q_reci_dis_max,q_reci_dis_mean,u_diff_qa_days_sum,u_diff_qa_days_max,u_diff_qa_days_mean,q_diff_qa_days_sum,q_diff_qa_days_max,q_diff_qa_days_mean,qid_enc,uid_enc
0,Q2166419046,M401693808,3.0,0,1,0,0,0,1864,160,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73358,1508098
1,Q604029601,M2317670257,3.0,1,0,0,0,0,1347,193,...,NaN,NaN,2.0,2.0,1.000000,NaN,NaN,NaN,218946,657985
2,Q2443223942,M3544409350,2.0,1,0,0,0,0,499,191,...,0.0,0.0,NaN,NaN,NaN,305.0,13.0,9.531250,90646,1272353
3,Q795459266,M2818659842,4.0,1,0,0,0,0,1347,193,...,0.0,0.0,2260.0,1391.0,376.666667,667.0,234.0,222.333333,231007,909154
4,Q110462128,M848334644,3.0,1,0,0,0,0,1905,26,...,NaN,NaN,2156.0,1426.0,38.500000,NaN,NaN,NaN,6492,1856019


In [36]:
test.head()

,qid,uid,freq,uf_b1,uf_b2,uf_b3,uf_b4,uf_b5,uf_c1,uf_c2,...,q_reci_dis_max,q_reci_dis_mean,u_diff_qa_days_sum,u_diff_qa_days_max,u_diff_qa_days_mean,q_diff_qa_days_sum,q_diff_qa_days_max,q_diff_qa_days_mean,qid_enc,uid_enc
0,Q1493039281,M64135255,3.0,0,1,0,0,0,871,89,...,NaN,NaN,1087.0,1085.0,543.50,NaN,NaN,NaN,31007,1752471
1,Q2023398782,M2536956560,4.0,0,1,0,1,0,871,89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64399,767936
2,Q4151338694,M3294926344,3.0,1,0,0,0,0,871,89,...,NaN,NaN,1125.0,694.0,281.25,NaN,NaN,NaN,198729,1147979
3,Q3271436624,M3744310794,NaN,0,0,0,0,0,871,89,...,0.0,0.0,807.0,807.0,807.00,12.0,11.0,3.0,142876,1372677
4,Q3314287018,M1349051752,3.0,0,1,0,0,0,282,132,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145710,173892


In [37]:
data.head()

,day,describe_questions,focus_topics,freq,hour,interest_topic,label,q_ans_count,q_diff_qa_days_max,q_diff_qa_days_mean,...,uf_b3,uf_b4,uf_b5,uf_c1,uf_c2,uf_c3,uf_c4,uf_c5,uid,uid_enc
0,3865,[-0.01129505 3.07473125 -0.96356895 -1.014970...,[ 0.56766598 -0.29046254 0.4245674 1.052861...,3.0,22,[-0.26751407 -1.18381493 0.63383315 0.594649...,0.0,NaN,NaN,NaN,...,0,0,0,1864,160,244,787,1,M401693808,1508098
1,3862,[ 0.26690856 -0.83421576 -2.30800738 2.237093...,[ 0.32695651 -0.46650439 0.3626379 0.701971...,3.0,15,[ 1.59588447 -3.68648277 -1.00653933 0.097648...,0.0,NaN,NaN,NaN,...,0,0,0,1347,193,0,433,1,M2317670257,657985
2,3867,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,2.0,4,[ 0.79694882 0.73822912 0.17052762 -1.028728...,0.0,32.0,13.0,9.531250,...,0,0,0,499,191,175,695,1,M3544409350,1272353
3,3861,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[ 3.75784285 1.53372855 3.82858152 0.131264...,4.0,20,[ 0.85749254 -1.3819219 -0.06080027 -0.818199...,0.0,3.0,234.0,222.333333,...,0,0,0,1347,193,0,433,1,M2818659842,909154
4,3862,[ 4.27634221 -1.3773608 -0.08570212 -0.317267...,[ 3.31510868 -2.39552719 -1.72441314 1.528787...,3.0,8,[ 0.83324934 -0.77533439 -1.34746479 0.467904...,1.0,NaN,NaN,NaN,...,0,0,0,1905,26,369,1237,1,M848334644,1856019


In [38]:
data.columns.values.tolist()

['day',
 'describe_questions',
 'focus_topics',
 'freq',
 'hour',
 'interest_topic',
 'label',
 'q_ans_count',
 'q_diff_qa_days_max',
 'q_diff_qa_days_mean',
 'q_diff_qa_days_sum',
 'q_has_img_max',
 'q_has_img_mean',
 'q_has_img_sum',
 'q_has_video_max',
 'q_has_video_mean',
 'q_has_video_sum',
 'q_is_dest_max',
 'q_is_dest_mean',
 'q_is_dest_sum',
 'q_is_good_max',
 'q_is_good_mean',
 'q_is_good_sum',
 'q_is_rec_max',
 'q_is_rec_mean',
 'q_is_rec_sum',
 'q_reci_cheer_max',
 'q_reci_cheer_mean',
 'q_reci_cheer_sum',
 'q_reci_comment_max',
 'q_reci_comment_mean',
 'q_reci_comment_sum',
 'q_reci_dis_max',
 'q_reci_dis_mean',
 'q_reci_dis_sum',
 'q_reci_mark_max',
 'q_reci_mark_mean',
 'q_reci_mark_sum',
 'q_reci_no_help_max',
 'q_reci_no_help_mean',
 'q_reci_no_help_sum',
 'q_reci_tks_max',
 'q_reci_tks_mean',
 'q_reci_tks_sum',
 'q_reci_uncheer_max',
 'q_reci_uncheer_mean',
 'q_reci_uncheer_sum',
 'q_reci_xxx_max',
 'q_reci_xxx_mean',
 'q_reci_xxx_sum',
 'q_word_count_max',
 'q_word_co

In [39]:
# count编码
count_fea = ['uid_enc', 'qid_enc', 'freq', 'uf_c1', 'uf_c2', 'uf_c3', 'uf_c4', 'uf_c5']
for feat in count_fea:
    col_name = '{}_count'.format(feat)
    data[col_name] = data[feat].map(data[feat].value_counts().astype(int))
    data.loc[data[col_name] < 2, feat] = -1
    data[feat] += 1
    data[col_name] = data[feat].map(data[feat].value_counts().astype(int))
    data[col_name] = (data[col_name] - data[col_name].min()) / (data[col_name].max() - data[col_name].min())
    # 

# 问题被回答的次数



In [40]:
# 压缩数据
t = data.dtypes
for x in t[t == 'int64'].index:
    data[x] = data[x].astype('int32')

for x in t[t == 'float64'].index:
    data[x] = data[x].astype('float32')

data['wk'] = data['day'] % 7
data_1 = data.drop(['describe_questions','focus_topics','interest_topic','question_topics','title_questions'], axis=1)
data_1.head()
feature_cols = [x for x in data_1.columns if x not in ('label', 'uid', 'qid', 'dt', 'day')]

In [41]:
data.head()

,day,describe_questions,focus_topics,freq,hour,interest_topic,label,q_ans_count,q_diff_qa_days_max,q_diff_qa_days_mean,...,uid_enc,uid_enc_count,qid_enc_count,freq_count,uf_c1_count,uf_c2_count,uf_c3_count,uf_c4_count,uf_c5_count,wk
0,3865,[-0.01129505 3.07473125 -0.96356895 -1.014970...,[ 0.56766598 -0.29046254 0.4245674 1.052861...,4.0,22,[-0.26751407 -1.18381493 0.63383315 0.594649...,0.0,NaN,NaN,NaN,...,1508099,0.000000,0.000000,0.963521,0.000177,0.003324,0.002225,0.037172,1.0,1
1,3862,[ 0.26690856 -0.83421576 -2.30800738 2.237093...,[ 0.32695651 -0.46650439 0.3626379 0.701971...,4.0,15,[ 1.59588447 -3.68648277 -1.00653933 0.097648...,0.0,NaN,NaN,NaN,...,657986,0.000000,1.000000,0.963521,0.000232,0.001248,0.446553,0.000395,1.0,5
2,3867,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,3.0,4,[ 0.79694882 0.73822912 0.17052762 -1.028728...,0.0,32.0,13.0,9.531250,...,1272354,0.000000,0.000648,0.305479,0.000048,0.001116,0.031516,0.000425,1.0,3
3,3861,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[ 3.75784285 1.53372855 3.82858152 0.131264...,5.0,20,[ 0.85749254 -1.3819219 -0.06080027 -0.818199...,0.0,3.0,234.0,222.333328,...,0,1.000000,1.000000,1.000000,0.000232,0.001248,0.446553,0.000395,1.0,4
4,3862,[ 4.27634221 -1.3773608 -0.08570212 -0.317267...,[ 3.31510868 -2.39552719 -1.72441314 1.528787...,4.0,8,[ 0.83324934 -0.77533439 -1.34746479 0.467904...,1.0,NaN,NaN,NaN,...,1856020,0.000029,0.000000,0.963521,0.000072,0.000077,0.003912,0.001141,1.0,5


In [42]:
data_1.head()

,day,freq,hour,label,q_ans_count,q_diff_qa_days_max,q_diff_qa_days_mean,q_diff_qa_days_sum,q_has_img_max,q_has_img_mean,...,uid_enc,uid_enc_count,qid_enc_count,freq_count,uf_c1_count,uf_c2_count,uf_c3_count,uf_c4_count,uf_c5_count,wk
0,3865,4.0,22,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1508099,0.000000,0.000000,0.963521,0.000177,0.003324,0.002225,0.037172,1.0,1
1,3862,4.0,15,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,657986,0.000000,1.000000,0.963521,0.000232,0.001248,0.446553,0.000395,1.0,5
2,3867,3.0,4,0.0,32.0,13.0,9.531250,305.0,1.0,0.062500,...,1272354,0.000000,0.000648,0.305479,0.000048,0.001116,0.031516,0.000425,1.0,3
3,3861,5.0,20,0.0,3.0,234.0,222.333328,667.0,1.0,0.333333,...,0,1.000000,1.000000,1.000000,0.000232,0.001248,0.446553,0.000395,1.0,4
4,3862,4.0,8,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1856020,0.000029,0.000000,0.963521,0.000072,0.000077,0.003912,0.001141,1.0,5


In [43]:
logging.info("feature size %s", len(feature_cols))

X_train_all = data_1.iloc[:len(train_label)][feature_cols]
y_train_all = data_1.iloc[:len(train_label)]['label']
test = data.iloc[len(train_label):]

# del data
assert len(test) == sub_size

logging.info("train shape %s, test shape %s", train_label.shape, test.shape)

fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for index, (train_idx, val_idx) in enumerate(fold.split(X=X_train_all, y=y_train_all)):
    break

X_train, X_val, y_train, y_val = X_train_all.iloc[train_idx][feature_cols], X_train_all.iloc[val_idx][feature_cols], \
                                 y_train_all.iloc[train_idx], \
                                 y_train_all.iloc[val_idx]
del X_train_all

model_lgb = LGBMClassifier(n_estimators=2000, n_jobs=-1, objective='binary', seed=1000, silent=True)
model_lgb.fit(X_train, y_train,
              eval_metric=['logloss', 'auc'],
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=100)

sub['label'] = model_lgb.predict_proba(test[feature_cols])[:, 1]


sub.to_csv('../example/result_1208.txt', index=None, header=None, sep='\t')

[2019-12-08 18:26:36,270] INFO in <ipython-input-43-2c4e7348f3d2>: feature size 116
[2019-12-08 18:26:36,276] INFO in <ipython-input-43-2c4e7348f3d2>: train shape (10195, 116), test shape (1141683, 125)


[1]	valid_0's auc: 0.672209	valid_0's binary_logloss: 0.437258
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.681677	valid_0's binary_logloss: 0.431908
[3]	valid_0's auc: 0.689474	valid_0's binary_logloss: 0.427541
[4]	valid_0's auc: 0.690732	valid_0's binary_logloss: 0.424504
[5]	valid_0's auc: 0.698281	valid_0's binary_logloss: 0.420863
[6]	valid_0's auc: 0.704432	valid_0's binary_logloss: 0.417528
[7]	valid_0's auc: 0.70674	valid_0's binary_logloss: 0.415386
[8]	valid_0's auc: 0.706498	valid_0's binary_logloss: 0.414028
[9]	valid_0's auc: 0.710849	valid_0's binary_logloss: 0.411554
[10]	valid_0's auc: 0.708847	valid_0's binary_logloss: 0.410641
[11]	valid_0's auc: 0.709087	valid_0's binary_logloss: 0.409961
[12]	valid_0's auc: 0.710788	valid_0's binary_logloss: 0.408746
[13]	valid_0's auc: 0.709948	valid_0's binary_logloss: 0.407848
[14]	valid_0's auc: 0.710949	valid_0's binary_logloss: 0.407331
[15]	valid_0's auc: 0.713025	valid_0's binary_loglos